# Building a QSAR model using the Morgan fingerprint in rdkit and Random forest from Spark's Machine Learning

## QSAR
We know that the structure of a molecule determines the molecule's properties, but we are not capable of calculating molecular properties from molecular structures in general. One approach to doing this is used in the field of Quantitative Structure-Activity Relationships (QSAR) / Quantitative Structure-Property Relationship (QSPR), where a molecular structure is mathematically described and then machine learning is used to predict molecular activity or molecular properties. The names QSAR and QSPR are sometimes used a bit sloppily and mixed up, QSAR seems to be the most common term but QSPR is arguably a bit more general. So we need a strategy for describing the molecular properties, a machine larning algorithm, and a lot of data that we can use to train our machine larning algorithm on.

### Molecular descriptor
A molecular descriptor is a mathematical representation of a molecule resulting from the transformation of the symbolic representation of a molecule into numbers. One approach might simply be to count the number of different atoms or fragments. There are many different approaches, some calculated from the matrix resulting from the pairwise distances between all atoms in the molecule, some being fragment based. The idea is to create a vector of numbers calculated in such a way that similar compounds will have similar vectors. There are two different approaches worth mentioning. Either each posiition in the vector is predefined and map to one thing, meaning that we know exactly what each position means, (_e.g._, if a position in the vector has the number 1 we know that the molecule contains exactly the substructure that corresponds to that position), or a _hashing_ algorithm is used to go from a descriptor to a position in the vector. The hashing algorithm is a one-way algorithm that turns, _e.g._, a string into a number (corresponding to the position in the vector). The hashing approach has the benefit that it does not require someone to predefine which structures should be used, anything found can be hashed and used in the vector but we can not easily know what each position means, and there is the risk for hash collisions (_i.e._, the hashing algorithm mapping many things into the same number making it impossible to say which one it originally was). In this lab we will use a circular fingerpint known as the Morgan fingerprint as implemented in the Python library RDKit and hash it down to a vector. A circular fingerprint uses each atom in a molecular and describe the atom neighbours out to a certain distance or "radius".

### Machine learning algorithm
We will use the Random Forest algorithm, which is a good algorithm to start with. We will not go into details on how the algorithm works in this lab, but it constructs a multitude of decission trees and then weights them together.

### Dataset
The data we will look at in this lab is distribution coefficient, log D, we will use a calculated value that we have extracted from a database where this number has been calculated for many substances. Since we want to work with big data we will use a calculated value. Of course the quality of the model depends heavily on the number of training examples.

Let's start by looking at the first lines of our datafile:

It is a tab separated file where the first line contains names for the columns and then follows SMILES and log D for all substances, one substance per line.

For dealing with the chemistry we will use [RDKit](http://www.rdkit.org/):

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

Next, we need to import the `SparkSession` and for the machine learning we will use `pyspark.ml`:

In [ ]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors

Finally, we will use a `numpy` array to convert from the RDKit representation of our dataset to the PySPARK representation of our dataset: 

In [ ]:
import numpy as np

First we need to get a `SparkSession` which will work as our connection to the Spark cluster:

In [ ]:
spark = SparkSession\
        .builder\
        .master("spark://192.168.2.32:7077") \
        .appName("RF_on_MorganFP")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

I have placed the dataset file on the Hadoop file system accessible for our script at: `hdfs://192.168.2.32:9000/acd_logd.smiles`. The dataset file starts with a header line and has the data in tab separated columns:

In [ ]:
df = spark.read.option("header","true")\
               .option("delimiter", '\t').csv("hdfs://192.168.2.32:9000/acd_logd.smiles")

Let's look at a couple of rows:

In [ ]:
df.head(5)

And count the number of lines:

In [ ]:
df.count()

It is quite many mulecular structures in this dataset and since we don't have so much time we will sample a fraction of it:

In [ ]:
fraction = 0.01

df_sample = df.sample(fraction)
df_sample.count()

Now we will create the input the to machine learning algorithm. In the end we want to have a dataframe with the logD values and the calculated molecular descriptor. The descriptor will consist of 2048 columns. In order to run RDKit in the Spark workers we will use the RDD representation and map it over to RDKit molecules:

In [ ]:
chemicals = df_sample.select("canonical_smiles", "acd_logd").rdd\
                     .map( lambda row: (row.canonical_smiles, float(row.acd_logd)) )\
                     .map( lambda x: (Chem.MolFromSmiles(x[0]), x[1]) )

Notice that we are hauling around the logd value in a tuple since we need to have it in the final step and since all data is spread out in the cluster we can not easily just add a column. Next we calculate the Morgan fingerprint for the molecules using 1024 bits:

In [ ]:
fingerprints = chemicals.map( lambda x: (AllChem.GetMorganFingerprintAsBitVect(x[0], 2, nBits=1024), x[1]) )

As I already mentioned, we will use a Numpy `array` in order to convert the fingerprints (tupled with the logD values) in a dense PySPARK vector and then to a dataframe:

In [ ]:
data = fingerprints.map( lambda x: (np.array(x[0]),x[1]) )\
                   .map( lambda x: (Vectors.dense(x[0].tolist()),x[1]) )\
                   .toDF(["features", "label"])
data.head(2)

Next we will run random forest in Spark. We will specify how to index the features while automatically identifying categorical features by saying anything more than 4 distinct values as continous (our fingperints ahve two values, 0 or 1). We will also split the data into training and test set and specify a random forest model:

In [ ]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# 30% for test set
(trainingData, testData) = data.randomSplit([0.7, 0.3])

rf = RandomForestRegressor(featuresCol="indexedFeatures")

Next we will chain the `featureIndexer` and the `RandomForestRegressor` using a `pyspark.ml.Pipeline` and train the model:

In [ ]:
pipeline = Pipeline(stages=[featureIndexer, rf])
model = pipeline.fit(trainingData)

Now, in order to say something about how good our model is we will predict log D for the molecules in our test set:

In [ ]:
predictions = model.transform(testData)
predictions.show(5)

Finally we will calculate Root Mean Squared Error (RMSE):

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


and stop our Spark session:

In [ ]:
spark.stop()

Of course, if you have the time you could try to sample a larger part of the dataset and see if you can get a better RMSE, truth be told this one is maybe not so impressive...